In [1]:
# -*- coding: utf-8 -*-
# HorseVsPeople


import os
from PIL import Image
import numpy as np
import tensorflow as tf


# http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip unzip and add to "./train_data"
# http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip unzip and add to "./val_data"

!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
#
TRAIN_DIR = "./train_data"
VAL_DIR = "./val_data"
!unzip -q horse-or-human.zip -d $TRAIN_DIR
!unzip -q validation-horse-or-human.zip -d $VAL_DIR

TRAIN_DIR = "./train_data"
VAL_DIR = "./val_data"

--2022-05-05 07:51:16--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 173.194.193.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   156MB/s    in 0.9s    

2022-05-05 07:51:17 (156 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2022-05-05 07:51:17--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 108.177.111.128, 142.250.128.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (1

In [2]:
train_sample_fn = os.listdir(os.path.join(TRAIN_DIR, "horses"))[0]
train_sample_path = os.path.join(TRAIN_DIR, "horses", train_sample_fn)

img = Image.open(train_sample_path)
img_data = np.asarray(img)

print("Image shape:", img_data.shape)
print("Pixel in range:", np.min(img_data), np.max(img_data))

print(img_data[:1])


img_size = (150, 150)
img_shape = (*img_size, 3)    # We didn't include transparency channel

Image shape: (300, 300, 4)
Pixel in range: 1 255
[[[ 24  30  28 255]
  [ 28  36  37 255]
  [ 34  48  56 255]
  ...
  [ 87  77  72 255]
  [ 74  70  66 255]
  [ 50  57  64 255]]]


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def gen_new_data(data_folder):
  data_gen = ImageDataGenerator(rescale=1./255)
  data = data_gen.flow_from_directory(
      data_folder,
      target_size=img_size,
      batch_size=64,
      class_mode="binary",
  )
  return data

train_data_gen = gen_new_data(TRAIN_DIR)
val_data_gen = gen_new_data(VAL_DIR)


from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [4]:
import tensorflow as tf

model_2 = Sequential([
  Conv2D(16, 3, activation='elu', input_shape=(150, 150, 3)),
  BatchNormalization(),
  MaxPooling2D(),
  Conv2D(32, 3, activation='elu'),
  MaxPooling2D(),
  tf.keras.layers.SpatialDropout2D(0.6),
  Conv2D(64, 3, activation='elu'),
  MaxPooling2D(),
  Conv2D(128, 3, activation='elu'),
  MaxPooling2D(),
  Flatten(),
  Dense(512, activation='elu'),
  Dense(1, activation='sigmoid')
])

# Compile the model
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

#callback=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5) , callbacks=[callback]

# Fit the model
model_2.fit(train_data_gen, epochs=20, validation_data=val_data_gen)

model_2.evaluate(val_data_gen)

Epoch 1/20
17/17 [==============================] - 18s 452ms/step - loss: 1.7091 - accuracy: 0.7702 - val_loss: 1.1067 - val_accuracy: 0.7070
Epoch 2/20
17/17 [==============================] - 7s 439ms/step - loss: 0.1636 - accuracy: 0.9435 - val_loss: 0.1672 - val_accuracy: 0.9375
Epoch 3/20
17/17 [==============================] - 7s 438ms/step - loss: 0.1044 - accuracy: 0.9572 - val_loss: 0.1884 - val_accuracy: 0.9414
Epoch 4/20
17/17 [==============================] - 7s 430ms/step - loss: 0.0786 - accuracy: 0.9747 - val_loss: 0.3392 - val_accuracy: 0.8945
Epoch 5/20
17/17 [==============================] - 7s 431ms/step - loss: 0.0473 - accuracy: 0.9834 - val_loss: 0.1372 - val_accuracy: 0.9492
Epoch 6/20
17/17 [==============================] - 8s 442ms/step - loss: 0.0271 - accuracy: 0.9932 - val_loss: 0.1767 - val_accuracy: 0.9453
Epoch 7/20
17/17 [==============================] - 7s 419ms/step - loss: 0.0183 - accuracy: 0.9951 - val_loss: 0.2324 - val_accuracy: 0.9336
Epoch

[0.6897506713867188, 0.8671875]

In [5]:
model_2.save('e3_q2.h5')